In [35]:
import pymysql
from pymysql.err import MySQLError
import requests
import time
import os
import mimetypes
import json
from os import listdir
from os.path import isfile, join

def connect_to_db():
    return pymysql.connect(
            host='localhost',
            user='mysql',
            password='mysql',
            database='mysql',
            port=3306
        )

def fetch_new_images_post(cursor):
    query = """
        SELECT post_id, link_images, link_videos
        FROM posts_info
    """
    cursor.execute(query)
    return cursor.fetchall()

def download_image(db_connection, cursor, url, post_id, download_path):
    downloaded = [os.path.splitext(f)[0] for f in listdir('contents') if isfile(join('contents', f))]
    if not post_id in downloaded:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            content_type = response.headers['Content-Type']
            extension = mimetypes.guess_extension(content_type) or '.jpg'
            
            filename = str(post_id) + extension
            filepath = os.path.join(download_path, filename)
    
            with open(filepath, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

In [ ]:
while True:
    db_connection = connect_to_db()
    cursor = db_connection.cursor()
    new_images = fetch_new_images_post(cursor)
    print(f"TO DOWNLOAD: {len(new_images)}")
    for post_id, link_images, link_videos in new_images:
        images = json.loads(link_images)
        videos = json.loads(link_videos)
        if images is not None:
            for image in images:
                try:
                    download_image(db_connection,cursor,image,post_id,'contents')
                except Exception as e:
                    continue
        if videos is not None:
            for video in videos:
                try:
                    download_image(db_connection,cursor,video,post_id,'contents')
                except Exception as e:
                    continue
    cursor.close()
    db_connection.close()

TO DOWNLOAD: 190
